- https://arxiv.org/pdf/2401.17270.pdf
- https://github.com/AILab-CVC/YOLO-World
- https://huggingface.co/spaces/stevengrove/YOLO-World
- https://colab.research.google.com/drive/1AmhbXBmH2MnJA8_aQ5EDoormQ61xlzdp#scrollTo=YQ86d81wVGMe
- https://twitter.com/skalskip92/status/1754916529672438173

In [ ]:
!pip install mmyolo

In [ ]:
!git clone https://github.com/onuralpszr/YOLO-World.git -b collab_friendly

In [ ]:
%cd YOLO-World/
!git branch

In [ ]:
!pip install tqdm~=4.65.0

In [ ]:
!python setup.py build develop

In [ ]:
%pip install -U openmim
!mim install "mmengine>=0.7.0"
!mim install "mmcv"

In [ ]:
!pip install supervision==0.18.0

In [ ]:
quit()

In [ ]:
import mmengine
import yolo_world
import mmyolo